<a href="https://colab.research.google.com/github/RuslanMavlitov/IDE/blob/master/Practice_ML_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика по рекомендательным системам

В этом модуле мы обсудили, как используются рекомендательные системы, как получают данные для них и как оценивают результаты работы РС. Мы начали знакомиться с алгоритмами построения систем рекомендаций и пока успели изучить более подробно один из них — РС на основе популярности. В этом юните мы построим систему рекомендаций, основываясь именно на этом методе. Вы сможете усовершенствовать её в следующем модуле, после того как освоите другие алгоритмы.

Для начала загрузим датасет "Articles sharing and reading from CI&T DeskDrop", включающий в себя собранные за один год логи DeskDrop — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
import math

Данные включают в себя два файла:

shared_articles.csv;

users_interactions.csv.

Начнём работать с файлом shared_articles.csv. 
Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

Для временной метки существует два возможных типа событий:

CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;

CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.

Для простоты мы рассматриваем здесь только тип события CONTENT SHARED.

In [2]:
shared_articles = pd.read_csv('/content/drive/MyDrive/SF/shared_articles.zip')
shared_articles.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


##Задание 6.1

Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [4]:
print(f'Размер датасета до среза по CONTENT SHARED {shared_articles.shape[0]}')
shared_articles_shared = shared_articles[shared_articles['eventType'] == 'CONTENT SHARED']
print(f'Размер датасета до среза после CONTENT SHARED {shared_articles_shared.shape[0]}')

Размер датасета до среза по CONTENT SHARED 3122
Размер датасета до среза после CONTENT SHARED 3047


Теперь откроем второй файл — users_interactions.csv.

В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

VIEW — просмотр,

LIKE — лайк,

COMMENT CREATED — комментарий,

FOLLOW — подписка,

BOOKMARK — добавление в закладки.

В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:

In [3]:
users_interactions = pd.read_csv('/content/drive/MyDrive/SF/users_interactions.zip')
users_interactions.head(3)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN


In [6]:
users_interactions['eventType'].value_counts()

VIEW               61086
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [5]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

##Задание 6.2

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [11]:
users_interactions['eventType_num'] = users_interactions['eventType'].replace(event_type)
print(round(users_interactions['eventType_num'].mean(), 2))
users_interactions['eventType_num'].value_counts()

1.24


1.0    61086
2.0     5745
2.5     2463
4.0     1611
3.0     1407
Name: eventType_num, dtype: int64

##Задание 6.3

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [14]:
users_df = (
    users_interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
 
users_5_df = users_df[users_df >= 5].reset_index()[['personId']]
print(len(users_5_df))

1140


##Задание 6.4

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [19]:
int_from_users5_df = users_interactions.loc[
    np.in1d(users_interactions.personId,
            users_5_df)] #этот метод используется, если мы хотим найти  пересечение двух одномерных массивов NumPy
print(int_from_users5_df.shape[0])

69868


Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

In [20]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

##Задание 6.5

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

In [28]:
full_df = (
    int_from_users5_df
    .groupby(['personId', 'contentId']).eventType_num.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
full_df['last_timestamp'] = (
    int_from_users5_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
round(full_df['last_timestamp'].mean(), 2)

1470587338.35

Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

##Задание 6.6

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [29]:
from sklearn.model_selection import train_test_split
 
split_ts = 1475519545
train_df = full_df.loc[full_df.last_timestamp < split_ts].copy()
test_df = full_df.loc[full_df.last_timestamp >= split_ts].copy()
 
print(len(train_df))

29329


Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [30]:
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]

##Задание 6.7

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [31]:
popular = (
    train_df
    .groupby('contentId')
    .eventType_num.sum().reset_index()
    .sort_values('eventType_num', ascending=False)
    .contentId.values
)
popular[0]

-6783772548752091658

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

##Задание 6.8

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления precision@10 воспользуйтесь следующей функцией:

In [33]:
def precision(column):
    return (final_df.apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

Итоговый результат округлите до трёх знаков после точки-разделителя.

In [38]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
final_df.apply(lambda row: 
               len(set(row['true_test'])
               .intersection(set(row['prediction_popular'])) / min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()


KeyError: ignored